In [98]:
from time import perf_counter
from typing import Hashable
from random import choices
from string import ascii_letters
from tabulate import tabulate
from dataclasses import dataclass


class Tree:
    def __repr__(self) -> str:
        def rec_print(node, level=0):
            line = "" if level == 0 else "┕━━━━ "
            ret = "\t" * level + line + node.symb + "\n"

            for _, child in node.next.items():
                ret += rec_print(child, level + 1)

            return ret

        return rec_print(self.root)


# Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia
# drzewa (1p.)
class Node:
    def __init__(self, symb):
        self.next: dict[Hashable, Node] = {}
        self.symb: str = symb


class Trie(Tree):
    def __init__(self, string):
        self.root = Node("")
        self.__build_trie(string)

    def __build_trie(self, string):
        for i in range(len(string)):
            node = self.root
            for c in string[i:]:
                if c in node.next:
                    node = node.next[c]
                else:
                    child = Node(symb=str(c))
                    node.next[c] = child
                    node = child


# Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu
# (1p.)
class LinkNode(Node):
    def __init__(self, symb, link=None):
        super().__init__(symb)
        self.link: LinkNode = self if link is None else link


class OnlineTrie(Tree):
    def __init__(self, string):
        self.root = LinkNode("")
        self.__build_trie(string)

    def __build_trie(self, string):
        root = self.root
        deepest = LinkNode(symb=string[0], link=root)
        root.next[string[0]] = deepest

        for c in string[1:]:
            curr: LinkNode = deepest
            prev: LinkNode = None

            while c not in curr.next:
                new_node = LinkNode(symb=c)
                curr.next[c] = new_node

                if prev is not None:
                    prev.link = new_node

                prev = new_node
                curr = curr.link

            if curr is root and curr.next[c] == prev:
                prev.link = root
            else:
                prev.link = curr.next[c]

            deepest = deepest.next[c]


# Drzewo sufiksów - algorytm Ukkonena (3p).
class CompressedNode:
    def __init__(self, l, r, parent=None):
        self.l: int = l
        self.r: int = r

        self.parent: CompressedNode = parent
        self.link: CompressedNode = None
        self.next: dict[str, CompressedNode] = {}

    def switch(self, ch):
        if ch in self.next:
            return self.next[ch]
        return None

    def len(self):
        return self.r - self.l


@dataclass
class State:
    node: CompressedNode
    pos: int


class CompressedTrie:
    def __init__(self, string):
        self.root = CompressedNode(0, 0)
        self.string = string
        self.__build_trie(string)

    def __build_trie(self, string):
        n: int = len(string)
        root: CompressedNode = self.root
        state: State = State(root, 0)

        def traverse(st: State, l: int, r: int) -> State:
            while l < r:
                if st.pos == st.node.len():
                    st = State(st.node.switch(string[l]), 0)
                    if st.node == None:
                        return st
                else:
                    if string[st.node.l + st.pos] != string[l]:
                        return State(None, -1)
                    if r - l < st.node.len() - st.pos:
                        return State(st.node, st.pos + r - l)

                    l += st.node.len() - st.pos
                    st.pos = st.node.len()

            return st

        def split(st: State) -> CompressedNode:
            if st.pos == st.node.len():
                return st.node
            if st.pos == 0:
                return st.node.parent

            v = st.node
            new_node = CompressedNode(v.l, v.l + st.pos, v.parent)
            new_node.next[string[v.l + st.pos]] = v

            v.parent.next[string[v.l]] = new_node
            v.parent = new_node
            v.l += st.pos

            return new_node

        def suff_link(v: CompressedNode) -> CompressedNode:
            if v.link != None:
                return v.link

            if v.parent == None:
                return self.root

            u = suff_link(v.parent)
            v.link = split(
                traverse(
                    State(u, u.len()), v.l + (1 if v.parent is self.root else 0), v.r
                )
            )

            return v.link

        for pos, ch in enumerate(string):
            while True:
                new_state: State = traverse(state, pos, pos + 1)
                if new_state.node != None:
                    state = new_state
                    break

                mid = split(state)
                mid.next[ch] = CompressedNode(pos, n, mid)

                state.node = suff_link(mid)
                state.pos = state.node.len()
                if mid is root:
                    break

    def __repr__(self) -> str:
        def rec_print(root, level=0):
            line = "" if level == 0 else "┕━━━━ "
            ret = "\t" * level + line + self.string[root.l : root.r] + "\n"

            for _, child in root.next.items():
                ret += rec_print(child, level + 1)

            return ret

        return rec_print(self.root)


In [99]:
Trie("ababcd" + "$")



	┕━━━━ a
		┕━━━━ b
			┕━━━━ a
				┕━━━━ b
					┕━━━━ c
						┕━━━━ d
							┕━━━━ $
			┕━━━━ c
				┕━━━━ d
					┕━━━━ $
	┕━━━━ b
		┕━━━━ a
			┕━━━━ b
				┕━━━━ c
					┕━━━━ d
						┕━━━━ $
		┕━━━━ c
			┕━━━━ d
				┕━━━━ $
	┕━━━━ c
		┕━━━━ d
			┕━━━━ $
	┕━━━━ d
		┕━━━━ $
	┕━━━━ $

In [100]:
OnlineTrie("ababcd" + "$")



	┕━━━━ a
		┕━━━━ b
			┕━━━━ a
				┕━━━━ b
					┕━━━━ c
						┕━━━━ d
							┕━━━━ $
			┕━━━━ c
				┕━━━━ d
					┕━━━━ $
	┕━━━━ b
		┕━━━━ a
			┕━━━━ b
				┕━━━━ c
					┕━━━━ d
						┕━━━━ $
		┕━━━━ c
			┕━━━━ d
				┕━━━━ $
	┕━━━━ c
		┕━━━━ d
			┕━━━━ $
	┕━━━━ d
		┕━━━━ $
	┕━━━━ $

In [101]:
CompressedTrie("ababcd" + "$")



	┕━━━━ ab
		┕━━━━ abcd$
		┕━━━━ cd$
	┕━━━━ b
		┕━━━━ abcd$
		┕━━━━ cd$
	┕━━━━ cd$
	┕━━━━ d$
	┕━━━━ $

In [102]:
END = "$"
s1 = "bbb" + END
s2 = "aabbabd" + END
s3 = "ababcd" + END
s4 = "abaababaabaabaabab" + END
s5 = "".join(choices(ascii_letters, k=100)) + END
s6 = open("1997_714_head.txt", "r").read() + END

data = []

for i, s in enumerate([s1, s2, s3, s4, s5, s6]):
    data.append(["s" + str(i)])
    for tree in (Trie, OnlineTrie, CompressedTrie):
        b = perf_counter()
        t = tree(s)
        e = perf_counter()
        data[i].append("{:.3e}".format(e - b) + "s")

print(
    tabulate(
        data,
        headers=["String", "Trie", "OnlineTrie", "CompressedTrie"],
        tablefmt="rounded_grid",
    )
)


╭──────────┬────────────┬──────────────┬──────────────────╮
│ String   │ Trie       │ OnlineTrie   │ CompressedTrie   │
├──────────┼────────────┼──────────────┼──────────────────┤
│ s0       │ 1.270e-05s │ 1.660e-05s   │ 2.960e-05s       │
├──────────┼────────────┼──────────────┼──────────────────┤
│ s1       │ 1.800e-05s │ 2.280e-05s   │ 2.550e-05s       │
├──────────┼────────────┼──────────────┼──────────────────┤
│ s2       │ 1.210e-05s │ 1.870e-05s   │ 1.750e-05s       │
├──────────┼────────────┼──────────────┼──────────────────┤
│ s3       │ 2.140e-04s │ 9.030e-05s   │ 2.168e-04s       │
├──────────┼────────────┼──────────────┼──────────────────┤
│ s4       │ 3.609e-03s │ 6.070e-03s   │ 5.652e-04s       │
├──────────┼────────────┼──────────────┼──────────────────┤
│ s5       │ 7.773e+00s │ 6.922e+00s   │ 1.096e-02s       │
╰──────────┴────────────┴──────────────┴──────────────────╯
